In [5]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

import warnings
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
trainimgs = mnist.train.images
trainlabels = mnist.train.labels
testimgs = mnist.test.images
testlabels = mnist.test.labels
ntrain, ntest, dim, nclasses = trainimgs.shape[0], testimgs.shape[0], trainimgs.shape[1],trainlabels.shape[1]
ntrain, ntest, dim, nclasses

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


(55000, 10000, 784, 10)

In [2]:
diminput = 28
dimhidden = 128
dimoutput = nclasses
nsteps = 28
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput,dimhidden])),
    'out': tf.Variable(tf.random_normal([dimhidden,dimoutput]))
}
b = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}

In [8]:
def _RNN(_X, _W, _b, _nsteps, _name):
    # 1. Permute input from [batchsize, nsteps, diminput]
    #    => [nsteps, batchsize, diminput]
    _X = tf.transpose(_X, [1, 0, 2])
    # 2. Reshape input to [nsteps*batchsize, diminput]
    _X = tf.reshape(_X, [-1, diminput])
    # 3. Input layer => Hidden layer
    _H = tf.matmul(_X, _W['hidden'] + _b['hidden'])
    # 4. Splite data to 'nsteps' chunks. An i-th chunck indices i-th batch data
    _Hsplit = tf.split(_H, _nsteps, 0)
    # 5. Get LSTM's final output (_LSTM_O) and state (_LSTM_S)
    #   Both _LSTM_O and _LSTM_S consist of 'batchsize' elements
    #   Only _LSTM_O will be used to predict the output
    with tf.variable_scope(_name) as scope:
        
        scope.reuse_variables()
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(dimhidden, forget_bias=1.0)
        _LSTM_O, _LSTM_S = tf.nn.static_rnn(lstm_cell, _Hsplit, dtype = tf.float32)
    # 6. Output
    _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']
    # Return 
    return {
        'X': _X, 'H': _H, 'Hsplit': _Hsplit,
        'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 'O': _O
    }

In [9]:
learning_rate = 0.001
x = tf.placeholder('float', [None, nsteps, diminput])
y = tf.placeholder('float', [None, dimoutput])
myrnn = _RNN(x, weights, b, nsteps, 'basic')
pred = myrnn['O']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
accr = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(y,1)), tf.float32))
init = tf.global_variables_initializer()


In [11]:
training_epochs = 5
batch_size = 16
display_step = 1
#Session
sess = tf.Session()
sess.run(init)
# mini-batch learning
for epoch in range(training_epochs):
    avg_cost = 0.
    # total_batch = int(mnist.train.num_examples/batch_size)
    total_batch=100
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, nsteps, diminput))
        feeds={x:batch_xs, y:batch_ys}
        sess.run(optm,feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)/total_batch
    #display
    if epoch % display_step == 0:
        feeds_train = {x:batch_xs, y:batch_ys}
#         feeds_test = {x:mnist.test.images, y:mnist.test.labels, keepratio:1.0}
        train_acc = sess.run(accr, feed_dict=feeds_train)
#         test_acc = sess.run(accr, feed_dict = feeds_test)
        print("Epoch: %03d/%03d cost: %.9f train_acc: %.3f "%
             (epoch, training_epochs, avg_cost, train_acc))
print('Done')

Epoch: 000/005 cost: 1.605114360 train_acc: 0.750 
Epoch: 001/005 cost: 1.171595551 train_acc: 0.688 
Epoch: 002/005 cost: 0.999480328 train_acc: 0.688 
Epoch: 003/005 cost: 0.953950666 train_acc: 0.938 
Epoch: 004/005 cost: 0.924855090 train_acc: 0.812 
Done
